The grammar of µDhall is a much simplified version of the [Dhall grammar](https://github.com/dhall-lang/dhall-lang/blob/master/standard/dhall.abnf).

```
end-of-line =
      %x0A     ; "\n"
    / %x0D.0A  ; "\r\n"

valid-non-ascii = %x80-10FFFD

tab = %x09  ; "\t"

not-end-of-line = %x20-7F / valid-non-ascii / tab

line-comment = "--" *not-end-of-line end-of-line

whitespace-chunk =
      " "
    / tab
    / end-of-line
    / line-comment

whsp = *whitespace-chunk

; Nonempty whitespace.
whsp1 = 1*whitespace-chunk

; Uppercase or lowercase ASCII letter.
ALPHA = %x41-5A / %x61-7A

; ASCII digit.
DIGIT = %x30-39  ; 0-9

ALPHANUM = ALPHA / DIGIT

; A simple label cannot be one of the reserved keywords
; listed in the `keyword` rule.
; A PEG parser could use negative lookahead to
; enforce this, e.g. as follows:
; label =
;       keyword 1*simple-label-next-char
;     / !keyword (simple-label-first-char *simple-label-next-char)
label-first-char = ALPHA / "_"

label-next-char = ALPHANUM / "-" / "/" / "_"

label = label-first-char *label-next-char

; A nonreserved-label cannot be any of the reserved identifiers for builtins.
; Their list can be found in the `builtin` rule.
; The only place where this restriction applies is bound variables.
; A PEG parser could use negative lookahead to avoid parsing those identifiers,
; e.g. as follows:
; nonreserved-label =
;      builtin 1*label-next-char
;    / !builtin label
nonreserved-label = label

; Keywords.
let                   = "let"
in                    = "in"
forall-keyword        = "forall"
forall-symbol         = %x2200 ; Unicode FOR ALL: ∀
forall                = forall-symbol / forall-keyword

keyword = let / in / forall-keyword

; Builtin constants.
Natural = "Natural"
Natural-fold = "Natural/fold"
Natural-subtract = "Natural/subtract"
Type = "Type"
Kind = "Kind"

builtin =
    Natural
    / Natural-fold
    / Natural-subtract
    / Type / Kind

; Operators.
lambda        = %x3BB  / "\"
arrow         = %x2192 / "->"
plus          = "+"
times         = "*"

natural-literal =
    ; Decimal; leading 0 digits are not allowed
    / ("1" / "2" / "3" / "4" / "5" / "6" / "7" / "8" / "9") *DIGIT
    ; ... except for 0 itself
    / "0"

identifier = variable / builtin

variable = nonreserved-label [ whsp "@" whsp natural-literal ]

expression =
    ; "\(x : a) -> b"
      lambda whsp "(" whsp nonreserved-label whsp ":" whsp1 expression whsp ")" whsp arrow whsp expression
    ;
    ; "let x     = e1 in e2"
    ; We allow dropping the `in` between adjacent let-expressions; the following are equivalent:
    ; "let x = e1 let y = e2 in e3"
    ; "let x = e1 in let y = e2 in e3"
    / 1*let-binding in whsp1 expression
    ;
    ; "forall (x : a) -> b"
    / forall whsp "(" whsp nonreserved-label whsp ":" whsp1 expression whsp ")" whsp arrow whsp expression
    ;
    ; "a -> b" is shorthand syntax for "forall (_ : a) -> b"
    ;
    ; NOTE: Backtrack if parsing this alternative fails
    / operator-expression whsp arrow whsp expression
    ;
    ; "x : t"
    / annotated-expression

; Nonempty-whitespace to disambiguate `env:VARIABLE` from type annotations
annotated-expression = operator-expression [ whsp ":" whsp1 expression ]

; "let x = e1"
let-binding = let whsp1 nonreserved-label whsp "=" whsp expression whsp1

operator-expression = plus-expression

plus-expression = times-expression *(whsp plus whsp times-expression)

times-expression = application-expression *(whsp times whsp application-expression)

application-expression = primitive-expression *(whsp1 primitive-expression)

primitive-expression =
    ; 123
    natural-literal
    ; "x"
    ; "x@2"
    / identifier
    ;
    ; "( e )"
    / "(" whsp expression whsp ")"
```

When a grammar rule specifies `a / b` then `a` is preferred to `b`; if `a` and the rest are parsed successfully then the `b` variant is not considered.

We use the `fastparse` library to implement this grammar.

In [ ]:
object Grammar {
    import fastparse._
    import NoWhitespace._

/**
 * A fastparse implementation of the provided ABNF grammar,
 * resembling parts of the Dhall language.
 */
object DhallLikeParser {
  // --- Basic Character and String Parsers ---

  // end-of-line = %x0A / %x0D.0A
  def `end-of-line`[_: P] = P( "\n" | "\r\n" )

  // tab = %x09
  def `tab`[_: P] = P( "\t" )

  // valid-non-ascii = %x80-10FFFD
  // In fastparse, P(CharIn(min..max)) is used for character ranges.
  // We use `!end-of-line` combined with `AnyChar` to cover %x20-10FFFD
  // and then manually add the tab. This is simpler and more robust for
  // multi-byte characters in fastparse compared to raw code points.
  // The original ABNF:
  // not-end-of-line = %x20-7F / valid-non-ascii / tab

  // A simpler representation for not-end-of-line: any character that is not part of an end-of-line sequence.
  def `not-end-of-line`[_: P]: P[Unit] = P( !(`end-of-line`) ~ AnyChar )

  // line-comment = "--" *not-end-of-line end-of-line
  def `line-comment`[_: P] = P( "--" ~ `not-end-of-line`.rep ~ `end-of-line` )

  // whitespace-chunk = " " / tab / end-of-line / line-comment
  def `whitespace-chunk`[_: P] = P( " " | `tab` | `end-of-line` | `line-comment` )

  // whsp = *whitespace-chunk
  def `whsp`[_: P] = P( `whitespace-chunk`.rep )

  // whsp1 = 1*whitespace-chunk
  def `whsp1`[_: P] = P( `whitespace-chunk`.rep(1) )

  // --- Identifier and Keyword Parsers ---

  // ALPHA = %x41-5A / %x61-7A
  def `ALPHA`[_: P] = P( fastparse.CharIn("a-z", "A-Z") )

  // DIGIT = %x30-39
  def `DIGIT`[_: P] = P( fastparse.CharIn("0-9") )

  // ALPHANUM = ALPHA / DIGIT
  def `ALPHANUM`[_: P] = P( `ALPHA` | `DIGIT` )

  // label-first-char = ALPHA / "_"
  def `label-first-char`[_: P] = P( `ALPHA` | "_" )

  // label-next-char = ALPHANUM / "-" / "/" / "_"
  def `label-next-char`[_: P] = P( `ALPHANUM` | fastparse.CharIn("-", "/", "_") )

  // label = label-first-char *label-next-char
  def `label`[_: P] = P( `label-first-char` ~ `label-next-char`.rep )

  // Keywords
  def `let`[_: P] = P( "let" )
  def `in`[_: P] = P( "in" )
  def `forall-keyword`[_: P] = P( "forall" )
  def `forall-symbol`[_: P] = P( "\u2200" ) // Unicode FOR ALL: ∀
  def `forall`[_: P] = P( `forall-symbol` | `forall-keyword` )
  def `keyword`[_: P] = P( `let` | `in` | `forall-keyword` )

  // Builtin constants.
  def `Natural`[_: P] = P( "Natural" )
  def `Natural-fold`[_: P] = P( "Natural/fold" )
  def `Natural-subtract`[_: P] = P( "Natural/subtract" )
  def `Type`[_: P] = P( "Type" )
  def `Kind`[_: P] = P( "Kind" )
  def `builtin`[_: P] = P(
    `Natural-fold` | `Natural-subtract` | `Natural` | `Type` | `Kind` // Order matters for fastparse, longest match first
  )

  // nonreserved-label = label
  // The negative lookahead logic from ABNF is usually implemented by checking
  // for keywords *after* parsing the label in a post-processing step,
  // or by explicitly defining `nonreserved-label` to disallow keywords/builtins.
  // For simplicity and adhering strictly to the *parsed* grammar structure:
  def `nonreserved-label`[_: P] = P( `label` ) // Assuming semantic check for keywords/builtins is external

  // --- Literal and Variable Parsers ---

  // natural-literal = ("1" / ... / "9") *DIGIT / "0"
  def `natural-literal`[_: P] = P( fastparse.CharIn("1-9") ~ `DIGIT`.rep | "0" )

  // Operators.
  def `lambda`[_: P] = P( "\u03BB" | "\\" ) // Unicode lambda: λ
  def `arrow`[_: P] = P( "\u2192" | "->" ) // Unicode arrow: →
  def `plus`[_: P] = P( "+" )
  def `times`[_: P] = P( "*" )

  // variable = nonreserved-label [ whsp "@" whsp natural-literal ]
  def `variable`[_: P] = P( `nonreserved-label` ~ (`whsp` ~ "@" ~ `whsp` ~ `natural-literal`).? )

  // identifier = variable / builtin
  def `identifier`[_: P] = P( `builtin` | `variable` ) // Builtin must be tried first as it might be a prefix of a variable

  // --- Expression Parsers (Precedence climbing) ---

  // primitive-expression = natural-literal / identifier / "(" whsp expression whsp ")"
  def `primitive-expression`[_: P]: P[Unit] = P(
    `natural-literal`
      | `identifier`
      | "(" ~ `whsp` ~ `expression` ~ `whsp` ~ ")"
  )

  // application-expression = primitive-expression *(whsp1 primitive-expression)
  def `application-expression`[_: P]: P[Unit] = P( `primitive-expression` ~ (`whsp1` ~ `primitive-expression`).rep )

  // times-expression = application-expression *(whsp times whsp application-expression)
  def `times-expression`[_: P]: P[Unit] = P( `application-expression` ~ (`whsp` ~ `times` ~ `whsp` ~ `application-expression`).rep )

  // plus-expression = times-expression *(whsp plus whsp times-expression)
  def `plus-expression`[_: P]: P[Unit] = P( `times-expression` ~ (`whsp` ~ `plus` ~ `whsp` ~ `times-expression`).rep )

  // operator-expression = plus-expression
  def `operator-expression`[_: P]: P[Unit] = P( `plus-expression` )

  // annotated-expression = operator-expression [ whsp ":" whsp1 expression ]
  def `annotated-expression`[_: P]: P[Unit] = P( `operator-expression` ~ (`whsp` ~ ":" ~ `whsp1` ~ `expression`).? )

  // let-binding = let whsp1 nonreserved-label whsp "=" whsp expression whsp1
  def `let-binding`[_: P]: P[Unit] = P( `let` ~ `whsp1` ~ `nonreserved-label` ~ `whsp` ~ "=" ~ `whsp` ~ `expression` ~ `whsp1` )

  // expression is the entry point
  def `expression`[_: P]: P[Unit] = P(
    // \(x : a) -> b
    `lambda` ~ `whsp` ~ "(" ~ `whsp` ~ `nonreserved-label` ~ `whsp` ~ ":" ~ `whsp1` ~ `expression` ~ `whsp` ~ ")" ~ `whsp` ~ `arrow` ~ `whsp` ~ `expression`

    // 1*let-binding in whsp1 expression
    | `let-binding`.rep(1) ~ `in` ~ `whsp1` ~ `expression`

    // forall (x : a) -> b
    | `forall` ~ `whsp` ~ "(" ~ `whsp` ~ `nonreserved-label` ~ `whsp` ~ ":" ~ `whsp1` ~ `expression` ~ `whsp` ~ ")" ~ `whsp` ~ `arrow` ~ `whsp` ~ `expression`

    // operator-expression whsp arrow whsp expression (shorthand: a -> b)
    | `operator-expression` ~ `whsp` ~ `arrow` ~ `whsp` ~ `expression`

    // annotated-expression
    | `annotated-expression`
  )

  // Top level parser
  def `parser`[_: P] = P( `expression` ~ End )
}

object Example extends App {
  import fastparse.Parsed
  import DhallLikeParser._

  val testString =
    """
    let x = 1 + y@0 * 2 -- A comment
    let z = Natural in
    λ(a : Type) → ∀(b : Type) → a → b → a
    """

  println(s"Parsing test string:\n---$testString---\n")

  fastparse.parse(testString, parser(_)) match {
    case Parsed.Success(_, index) =>
      println(s"Parsing succeeded! Reached index: $index (Full length)")
    case Parsed.Failure(expected, index, extra) =>
      println(s"Parsing failed at index: $index")
      println(s"Expected: $expected")
      // Print context of the failure
      val pre = testString.substring(0, index)
      val post = testString.substring(index)
      println(s"Context:\n'${pre}█${post}'")
      println(s"Trace: ${extra.trace().longAggregate}")
  }
}
}